In [1]:
import imagej; import scyjava
scyjava.config.add_option('-Xmx6g') # 使用メモリ上限を必要に応じ変更

# インタラクティブモードで開く
ij = imagej.init('C:\Program Files\Fiji.app', mode='interactive')
ij.ui().showUI() # UI表示も行わせておくと最初はわかりやすい

def open_image(img_path):
    ij.py.run_macro("""open("{}");""".format(img_path))
    return ij.WindowManager.getCurrentWindow()

def image_to_Luminance(image_window):
    image_window.toFront()
    ij.py.run_macro("""run("RGB to Luminance");""")
    return ij.WindowManager.getCurrentWindow()

def dog_image(image_window, sigma):
    image_window.toFront()
    ij.py.run_macro("""run("16-bit");""")
    ij.py.run_macro("""run("Duplicate...", " ");""")
    ij.py.run_macro("""run("Gaussian Blur...", "sigma={}");""".format(sigma))
    blured_image = ij.WindowManager.getCurrentWindow()
    ij.py.run_macro("""imageCalculator("Difference create 32-bit","{}","{}");""".
                    format(image_window, blured_image))
    dog_image = ij.WindowManager.getCurrentWindow()
    ij.py.run_macro("""run("Enhance Local Contrast (CLAHE)", 
                    "blocksize=127 histogram=256 maximum=3 mask=*None*");""")
    ij.py.run_macro("""run("8-bit");""")    
    return ij.WindowManager.getCurrentWindow()

def ridge_detection(image_window, sigma):
    image_window.toFront()
    ij.py.run_macro("""run("Ridge Detection", "line_width=3.5 high_contrast=230 
      low_contrast=87 estimate_width extend_line displayresults add_to_manager 
      method_for_overlap_resolution=NONE sigma={} lower_threshold=3.06 
      upper_threshold=7.99 minimum_line_length=0 maximum=0");""".format(sigma))
    results = ij.ResultsTable.getResultsTable()
    ridges = {}
    for column in results.getHeadings():
        ridges[column] = [item for item in results.getColumn(column)]
    return ridges

# 処理を実行させる
src_img_window = open_image("sample.tif")
src_luminance_img_window = image_to_Luminance(src_img_window)
dog_img_window = dog_image(src_luminance_img_window, 15) # 鮮鋭化パラメータ
ridges = ridge_detection(dog_img_window, 2.2)            # 筋・畝抽出パラメータ

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

thetas = np.array(ridges['Angle of normal']); r = ridges['Length']
fig = plt.figure(figsize=(6, 6))
for idx, theta in enumerate(thetas):
    plt.polar([theta, theta+np.pi], [r[idx]/2, r[idx]/2],
              color='b',alpha=0.02,linestyle='-')